In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Define the neural network model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the training loop
def train(model, train_loader, criterion, optimizer, num_epochs):
    train_losses = []
    test_accs = []
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            inputs = inputs.view(inputs.size(0), -1)  # Flatten the input
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_losses.append(running_loss/len(train_loader))
        test_acc = test(model, test_loader)
        test_accs.append(test_acc)
        print('Epoch [{}/{}], Train Loss: {:.4f}, Test Acc: {:.4f}'.format(epoch+1, num_epochs, train_losses[-1], test_acc))
    return train_losses, test_accs

# Define the testing loop
def test(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.view(inputs.size(0), -1)  # Flatten the input
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = correct / total
    return acc

# Define the data
input_size = 784  # 28x28 pixels
hidden_size = 128
output_size = 10  # 10 classes (digits 0-9)
transform = transforms.Compose([transforms.ToTensor()])
train_data = datasets.MNIST(root='./data/', transform=transform, train=True, download=True)
test_data = datasets.MNIST(root='./data/', transform=transform, train=False, download=True)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

# Create the model, loss function, and optimizer
model = MLP(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model and plot the results
num_epochs = 10
train_losses, test_accs = train(model, train_loader, criterion, optimizer, num_epochs)
plt.plot(range(num_epochs), test_accs)
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy')
plt.show()

Epoch [1/10], Train Loss: 1.2941, Test Acc: 0.8622
Epoch [2/10], Train Loss: 0.5067, Test Acc: 0.8912
Epoch [3/10], Train Loss: 0.3989, Test Acc: 0.9013
Epoch [4/10], Train Loss: 0.3567, Test Acc: 0.9082


KeyboardInterrupt: 